<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [8]:
# FIXME

dfs = []
for url in archivos_anio:
    tmp = pd.read_csv(url)
    # normalizar nombres de columnas a minúscula (y limpiar espacios)
    tmp.columns = tmp.columns.str.lower().str.strip()
    dfs.append(tmp)

# (a) Unir por filas (ventanas de tiempo) → df_anio
df_anio = pd.concat(dfs, ignore_index=True)

# (b) Explorar códigos ISO y limpiar el caso inconsistente

dups = (
    df_codigos.groupby('codigo_iso')['pais']
    .nunique()
    .reset_index(name='n_paises')
)
dups = dups[dups['n_paises'] > 1]

print("\nCódigos ISO con más de un nombre de país asociado:")
if dups.empty:
    print("No se encontraron inconsistencias.")
else:
    print(dups)

    print("\nFilas involucradas en las inconsistencias:")
    print(df_codigos[df_codigos['codigo_iso'].isin(dups['codigo_iso'])].sort_values(['codigo_iso','pais']))

    df_codigos_clean = (
        df_codigos.assign(pais_len=df_codigos['pais'].astype(str).str.len())
        .sort_values(['codigo_iso','pais_len'], ascending=[True, False])
        .drop_duplicates(subset='codigo_iso', keep='first')
        .drop(columns='pais_len')
    )
    print("\nDespués de limpiar (conservando un único país por código_iso):")
    print(df_codigos_clean[df_codigos_clean['codigo_iso'].isin(dups['codigo_iso'])])

# If no duplicates were found, use the original df_codigos
if dups.empty:
    df_codigos_clean = df_codigos.copy()


# (c) Unir df_anio con df_codigos por 'codigo_iso' → df
#     Mantener solo coincidencias (inner)

if 'codigo_iso' not in df_anio.columns:
    raise KeyError("En df_anio no existe la columna 'codigo_iso'. Revisa los CSV para confirmar el nombre exacto.")

df = pd.merge(df_anio, df_codigos_clean, on='codigo_iso', how='inner')

print("\nDataFrame final df →", df.shape)
print(df.head(10))


Códigos ISO con más de un nombre de país asociado:
    codigo_iso  n_paises
179        ZWE         2

Filas involucradas en las inconsistencias:
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo

Después de limpiar (conservando un único país por código_iso):
    codigo_iso      pais
179        ZWE  Zimbabue

DataFrame final df → (3060, 5)
  codigo_iso  anio  indice  ranking                    pais
0        AFG  2001    35.5     59.0             Afghanistán
1        AGO  2001    30.2     50.0                  Angola
2        ALB  2001     NaN      NaN                 Albania
3        AND  2001     NaN      NaN                 Andorra
4        ARE  2001     NaN      NaN  Emiratos Árabes Unidos
5        ARG  2001    12.0      8.0               Argentina
6        ARM  2001     NaN      NaN                 Armenia
7        ATG  2001     NaN      NaN       Antigua y Barbuda
8        AUS  2001     3.5     48.0               Australia
9        AUT  2001     7.5     86



### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [9]:
# FIXME

import pandas as pd
import numpy as np

print("\n" + "="*80)
print("2) EXPLORACIÓN INICIAL DEL CONJUNTO DE DATOS")
print("="*80)

# --------------------------------------------------------------------------------------
# 1)
# --------------------------------------------------------------------------------------
print("\n[1] ESTRUCTURA DEL DATAFRAME")
n_filas, n_cols = df.shape
print(f"- Filas (observaciones): {n_filas}")
print(f"- Columnas: {n_cols}")
print(f"- Nombres de columnas: {list(df.columns)}")

print("\nTipos de datos por columna:")
print(df.dtypes.sort_index())

# Heurística simple: ¿hay columnas 'object' que parecen fechas?
posibles_fechas = {}
for c in df.columns:
    if df[c].dtype == "object":
        parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
        ratio_parseable = parsed.notna().mean()
        if ratio_parseable > 0.6:  # si más del 60% se parsea como fecha, lo marcamos
            posibles_fechas[c] = ratio_parseable
if posibles_fechas:
    print("\nColumnas tipo 'object' que parecen fechas (proporción parseable):")
    for k, v in posibles_fechas.items():
        print(f"  - {k}: {v:.1%}")
else:
    print("\nNo se detectaron columnas 'object' que claramente sean fechas.")

# --------------------------------------------------------------------------------------
# 2)
# --------------------------------------------------------------------------------------
print("\n[2] RESUMEN ESTADÍSTICO")
print("\n.describe() (numéricas):")
print(df.describe(include=[np.number]))

for col in ["indice", "ranking"]:
    if col in df.columns:
        print(f"\nEstadísticos de '{col}':")
        print(f"  min: {df[col].min()}")
        print(f"  max: {df[col].max()}")
        print(f"  mean: {df[col].mean()}")

# Países con valores extremos en indice y ranking (maneja empates)
def extremos(col, etiqueta):
    if col not in df.columns:
        return
    vmin, vmax = df[col].min(), df[col].max()
    print(f"\nPaís(es) con {etiqueta} mínimo ({vmin}):")
    cols_show = [c for c in ["pais", "codigo_iso", col, "anio"] if c in df.columns]
    print(df.loc[df[col] == vmin, cols_show].drop_duplicates())
    print(f"\nPaís(es) con {etiqueta} máximo ({vmax}):")
    print(df.loc[df[col] == vmax, cols_show].drop_duplicates())

extremos("indice", "indice")
extremos("ranking", "ranking")

# --------------------------------------------------------------------------------------
# 3)
# --------------------------------------------------------------------------------------
print("\n[3] DATOS FALTANTES")
na_counts = df.isna().sum().sort_values(ascending=False)
print("\nValores nulos por columna:")
print(na_counts)

prop_faltantes = (df.isna().any(axis=1).mean())
print(f"\nProporción de filas con al menos un valor faltante: {prop_faltantes:.2%}")

cols_30 = (na_counts[na_counts > 0.30 * len(df)]).index.tolist()
if cols_30:
    print("\nColumnas con >30% de datos faltantes:")
    for c in cols_30:
        print(f"  - {c}: {na_counts[c]} nulos ({na_counts[c]/len(df):.2%})")
else:
    print("\nNo hay columnas con >30% de datos faltantes.")

# --------------------------------------------------------------------------------------
# 4)
# --------------------------------------------------------------------------------------
print("\n[4] UNICIDAD Y DUPLICADOS")
if "pais" in df.columns:
    print(f"- Países distintos (pais): {df['pais'].nunique()}")
else:
    print("- Columna 'pais' no encontrada.")

if "anio" in df.columns:
    print(f"- Años distintos (anio): {df['anio'].nunique()}")
else:
    print("- Columna 'anio' no encontrada.")

dup_count = df.duplicated().sum()
print(f"- Filas duplicadas exactamente iguales: {dup_count}")

# --------------------------------------------------------------------------------------
# 5)
# --------------------------------------------------------------------------------------
print("\n[5] VALIDACIÓN CRUZADA pais ↔ codigo_iso")

if {"pais", "codigo_iso"}.issubset(df.columns):
    # Un mismo código ISO mapea a >1 país
    g_iso = df.groupby("codigo_iso")["pais"].nunique().sort_values(ascending=False)
    inconsist_iso = g_iso[g_iso > 1]
    if inconsist_iso.empty:
        print("- No se encontraron códigos ISO asociados a más de un país.")
    else:
        print("\nCódigos ISO asociados a más de un país (conteos distintos):")
        print(inconsist_iso)
        print("\nDetalle de esas inconsistencias (muestras únicas):")
        print(
            df.loc[df["codigo_iso"].isin(inconsist_iso.index), ["codigo_iso", "pais"]]
              .drop_duplicates()
              .sort_values(["codigo_iso", "pais"])
              .head(100)  # limita el volcado
        )

    # Un mismo país con >1 código ISO (revisión inversa)
    g_pais = df.groupby("pais")["codigo_iso"].nunique().sort_values(ascending=False)
    inconsist_pais = g_pais[g_pais > 1]
    if not inconsist_pais.empty:
        print("\nPaíses asociados a más de un código ISO (conteos distintos):")
        print(inconsist_pais)
        print("\nDetalle inverso (muestras únicas):")
        print(
            df.loc[df["pais"].isin(inconsist_pais.index), ["pais", "codigo_iso"]]
              .drop_duplicates()
              .sort_values(["pais", "codigo_iso"])
              .head(100)
        )
else:
    print("- Faltan columnas 'pais' y/o 'codigo_iso' para validar consistencia.")


2) EXPLORACIÓN INICIAL DEL CONJUNTO DE DATOS

[1] ESTRUCTURA DEL DATAFRAME
- Filas (observaciones): 3060
- Columnas: 5
- Nombres de columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos por columna:
anio            int64
codigo_iso     object
indice        float64
pais           object
ranking       float64
dtype: object

No se detectaron columnas 'object' que claramente sean fechas.

[2] RESUMEN ESTADÍSTICO

.describe() (numéricas):
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000

Estadísticos de 'indice':
  min: 0.0
  max: 64536.0
  mean: 205.7823160660661

Estadísticos de 'ranking'

/tmp/ipython-input-3763587649.py:26: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-3763587649.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-3763587649.py:26: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_dateti




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [11]:
# respuesta
america = ['ARG','ATG','BLZ','BOL','BRA','CAN','CHL','COL','CRI',
           'CUB','DOM','ECU','GRD','GTM','GUY','HND','HTI','JAM',
           'MEX','NIC','PAN','PER','PRY','SLV','SUR','TTO','URY',
           'USA','VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

# --------------------------------------------------------------------------------------
# a)
# --------------------------------------------------------------------------------------

rows = []
for year in sorted(df_america['anio'].dropna().unique()):
    sub = df_america[df_america['anio'] == year].copy() # Use .copy() to avoid SettingWithCopyWarning

    # Find indices of min and max (handle cases with all NaN)
    # Explicitly set skipna=True to avoid FutureWarning
    i_min = sub['indice'].idxmin(skipna=True)
    i_max = sub['indice'].idxmax(skipna=True)

    # Check if indices are valid (not NaN) before accessing rows
    if pd.isna(i_min) or pd.isna(i_max):
        # If all values are NaN for this year, skip
        continue

    rmin = sub.loc[i_min]
    rmax = sub.loc[i_max]

    rows.append({
        'anio': year,
        'min_indice': rmin['indice'],
        'pais_min': rmin['pais'],
        'codigo_iso_min': rmin['codigo_iso'],
        'max_indice': rmax['indice'],
        'pais_max': rmax['pais'],
        'codigo_iso_max': rmax['codigo_iso'],
    })

res_for = pd.DataFrame(rows).sort_values('anio').reset_index(drop=True)
print("\nResultado con for:")
print(res_for)


# --------------------------------------------------------------------------------------
# b)
# --------------------------------------------------------------------------------------

# Indices (posiciones) de la fila con min y max por año
# Explicitly set skipna=True to avoid FutureWarning
idx_min = df_america.groupby('anio')['indice'].idxmin(skipna=True)
idx_max = df_america.groupby('anio')['indice'].idxmax(skipna=True)

# Drop NaN indices before using .loc
idx_min = idx_min.dropna()
idx_max = idx_max.dropna()


mins = (df_america.loc[idx_min, ['anio','pais','codigo_iso','indice']]
        .rename(columns={'pais':'pais_min',
                         'codigo_iso':'codigo_iso_min',
                         'indice':'min_indice'}))

maxs = (df_america.loc[idx_max, ['anio','pais','codigo_iso','indice']]
        .rename(columns={'pais':'pais_max',
                         'codigo_iso':'codigo_iso_max',
                         'indice':'max_indice'}))

# Merge the results, only keeping years present in both min and max (inner join)
res_groupby = (mins.merge(maxs, on='anio', how='inner')
                    .sort_values('anio')
                    .reset_index(drop=True))

print("\nResultado vectorizado (groupby + idxmin/idxmax):")
print(res_groupby)


Resultado con for:
    anio  min_indice           pais_min codigo_iso_min  max_indice   pais_max  \
0   2001        0.80             Canadá            CAN       90.30       Cuba   
1   2002        1.00  Trinidad y Tobago            TTO       97.83       Cuba   
2   2003        2.00  Trinidad y Tobago            TTO    35826.00  Argentina   
3   2004        2.00  Trinidad y Tobago            TTO       87.00       Cuba   
4   2005        4.50            Bolivia            BOL       95.00       Cuba   
5   2006        4.88             Canadá            CAN       96.17       Cuba   
6   2007        3.33             Canadá            CAN       88.33       Cuba   
7   2008        3.70             Canadá            CAN       94.00       Cuba   
8   2009        6.75     Estados Unidos            USA       78.00       Cuba   
9   2012        9.88            Jamaica            JAM       71.64       Cuba   
10  2013       10.90            Jamaica            JAM       70.92       Cuba   
11  2014

/tmp/ipython-input-3818776340.py:17: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  i_min = sub['indice'].idxmin(skipna=True)
/tmp/ipython-input-3818776340.py:18: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  i_max = sub['indice'].idxmax(skipna=True)


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [12]:
# FIX ME

tabla = pd.pivot_table(
    df,
    index="pais",
    columns="anio",
    values="indice",
    aggfunc="max",
    fill_value=0
)

print("\nTabla dinámica (primeras filas):")
print(tabla.head())

# ------------------------------------------------------------
# a)
# ------------------------------------------------------------
max_val = tabla.values.max()
min_val = tabla.replace(0, np.nan).min().min()

pais_max = tabla[tabla.eq(max_val).any(axis=1)].index.tolist()
pais_min = tabla[tabla.eq(min_val).any(axis=1)].index.tolist()

print(f"\n(a) Mayor valor de 'indice': {max_val}, País(es): {pais_max}")
print(f"(a) Menor valor (≠0): {min_val}, País(es): {pais_min}")

# ------------------------------------------------------------
# b)
# ------------------------------------------------------------
prom_anual = tabla.mean(axis=0)

anio_max = prom_anual.idxmax()
anio_min = prom_anual.idxmin()

print(f"\n(b) Año con promedio más alto: {anio_max} ({prom_anual.max():.2f})")
print(f"(b) Año con promedio más bajo: {anio_min} ({prom_anual.min():.2f})")

# ------------------------------------------------------------
# c)
# ------------------------------------------------------------
variabilidad = tabla.max(axis=1) - tabla.min(axis=1)
pais_var = variabilidad.idxmax()

print(f"\n(c) País con mayor variabilidad: {pais_var} ({variabilidad.max()})")

# ------------------------------------------------------------
# d)
# ------------------------------------------------------------
constantes = variabilidad[variabilidad == 0].index.tolist()
print(f"\n(d) Países con índice constante: {constantes}")

# ------------------------------------------------------------
# e)
# ------------------------------------------------------------
sin_datos = tabla[tabla.sum(axis=1) == 0].index.tolist()
print(f"\n(e) Países sin datos (todos 0): {sin_datos}")


Tabla dinámica (primeras filas):
anio         2001   2002   2003   2004   2005   2006   2007   2008   2009  \
pais                                                                        
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67   
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50   
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25   
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33   
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

anio          2012   2013   2014   2015   2017   2018   2019  
pais                                                          
Afghanistán  37.36  37.07  37.44  37.75  39.46  37.28  36.55  
Albania      30.88  29.92  28.77  29.92  29.92  29.49  29.84  
Alemania     10.24  10.23  11.47  14.80  14.97  14.39  14.60  
Algeria      36.54  36.26  36.63  41.69  42.83  43.13  45.75  
Andorra       6.82   6.82  19.87  19.87  21.03  